In [1]:
#import libraries and modules
import mne
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as  pd
import mne_bids
import string
import os
from os.path import join
from bids import BIDSLayout
sns.set(style="white", font_scale=1)

In [2]:
# Initiliaze the layout
data_path = os.path.join('/Users/alidzaye/rawdata')
layout = BIDSLayout(data_path)
print(layout)

/Users/alidzaye/opt/anaconda3/envs/mne/lib/python3.9/site-packages/bids/layout/models.py:148: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  warnings.warn("The 'extension' entity currently excludes the leading dot ('.'). "


BIDS Layout: .../Users/alidzaye/rawdata | Subjects: 8 | Sessions: 27 | Runs: 21


In [3]:
layout.get_subjects()

['001', '002', '003', '004', '005', '006', '007', '008']

In [4]:
layout.get_sessions()

['EphysMedOff01',
 'EphysMedOff02',
 'EphysMedOff03',
 'EphysMedOn01',
 'EphysMedOn02',
 'EphysMedOn03']

In [5]:
layout.get_tasks()

['BlockRotationL',
 'BlockRotationR',
 'BlockRotationWheel',
 'ContinuousStopping',
 'Evoked',
 'FreeDrawing',
 'ReadRelaxMoveL',
 'ReadRelaxMoveR',
 'Rest',
 'RestAndSpeech',
 'SelfpacedForceWheel',
 'SelfpacedRotationL',
 'SelfpacedRotationR',
 'SelfpacedSpeech',
 'Speech',
 'Transition',
 'UPDRSIII',
 'VigorStimL',
 'VigorStimR',
 'Visuomotor']

In [6]:
layout.get_acquisition()

['StimOff', 'StimOffOn', 'StimOn']

In [7]:
# Stim Rest data OFF & ON 
files_stimoff = layout.get(extension='vhdr', task='Rest', acquisition='StimOff', return_type='filename')
files_stimon = layout.get(extension='vhdr', task='Rest', acquisition='StimOn', return_type='filename')
files_stimoff_on = layout.get(extension='vhdr', task='Rest', acquisition='StimOffOn', return_type='filename')
print(len(files_stimoff))
print(len(files_stimon))
print(len(files_stimoff_on))

42
21
0


In [8]:
# Rest data OFF & ON StimOff
files = layout.get(extension='vhdr', task='Rest', acquisition='StimOff', return_type='filename')
files_off = layout.get(extension='vhdr', task='Rest', acquisition='StimOff', session=['EphysMedOff01','EphysMedOff02','EphysMedOff03'], return_type='filename')
files_on = layout.get(extension='vhdr', task='Rest', session=['EphysMedOn01','EphysMedOn02','EphysMedOn03'], acquisition='StimOff', return_type='filename')
print(len(files))
print(len(files_off))
print(len(files_on))

42
23
19


In [9]:
def read_files(files):
    '''
    reading Bids files
    '''
    raws=[]
    
    for index in range (len(files)):
        mne.io.read_raw(files[index])
        raws.append(mne.io.read_raw(files[index]))
        
    return raws

In [70]:
raws=read_files(files)

Extracting parameters from /Users/alidzaye/rawdata/sub-001/ses-EphysMedOff01/ieeg/sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-01_ieeg.vhdr...
Setting channel info structure...
Extracting parameters from /Users/alidzaye/rawdata/sub-001/ses-EphysMedOff01/ieeg/sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-01_ieeg.vhdr...
Setting channel info structure...
Extracting parameters from /Users/alidzaye/rawdata/sub-001/ses-EphysMedOff01/ieeg/sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-02_ieeg.vhdr...
Setting channel info structure...
Extracting parameters from /Users/alidzaye/rawdata/sub-001/ses-EphysMedOff01/ieeg/sub-001_ses-EphysMedOff01_task-Rest_acq-StimOff_run-02_ieeg.vhdr...
Setting channel info structure...
Extracting parameters from /Users/alidzaye/rawdata/sub-001/ses-EphysMedOn01/ieeg/sub-001_ses-EphysMedOn01_task-Rest_acq-StimOff_run-01_ieeg.vhdr...
Setting channel info structure...
Extracting parameters from /Users/alidzaye/rawdata/sub-001/ses-EphysMedOn01/ieeg/s

In [74]:
raws[3]

<RawBrainVision | sub-001_ses-EphysMedOn02_task-Rest_acq-StimOff_run-01_ieeg.eeg, 13 x 907572 (181.5 s), ~18 kB, data not loaded>

In [76]:
files_arr = np.array(files)

In [96]:
files[4]

'/Users/alidzaye/rawdata/sub-002/ses-EphysMedOff01/ieeg/sub-002_ses-EphysMedOff01_task-Rest_acq-StimOff_run-01_ieeg.vhdr'

In [27]:
#read the files with mne Bids
def read_BIDS_data(PATH_RUN, BIDS_PATH):
    """Given a run path and bids data path, read the respective data
    Parameters
    ----------
    PATH_RUN : string
    BIDS_PATH : string
    Returns
    -------
    raw_arr : mne.io.RawArray
    raw_arr_data : np.ndarray
    fs : int
    line_noise : int
    """
    entities = mne_bids.get_entities_from_fname(PATH_RUN)

    bids_path = mne_bids.BIDSPath(
            subject=entities["subject"],
            session=entities["session"],
            task=entities["task"],
            run=entities["run"],
            acquisition=entities["acquisition"],
            datatype="ieeg",
            root=BIDS_PATH,
                            )

    raw_arr = mne_bids.read_raw_bids(bids_path)
    return (
        raw_arr,
        raw_arr.get_data(),
        int(np.ceil(raw_arr.info["sfreq"])),
        int(raw_arr.info["line_freq"]),),

In [28]:
PATH_BIDS = r'/Users/alidzaye/rawdata'
PATH_RUN = r'/Users/alidzaye/rawdata/sub-003/ses-EphysMedOff01/ieeg/sub-003_ses-EphysMedOff01_task-Rest_acq-StimOff_run-01_ieeg.vhdr'

In [29]:
raw, dat, sfreq, line_freq = read_BIDS_data(PATH_RUN, PATH_BIDS)

Extracting parameters from /Users/alidzaye/rawdata/sub-003/ses-EphysMedOff01/ieeg/sub-003_ses-EphysMedOff01_task-Rest_acq-StimOff_run-01_ieeg.vhdr...
Setting channel info structure...
Reading channel info from /Users/alidzaye/rawdata/sub-003/ses-EphysMedOff01/ieeg/sub-003_ses-EphysMedOff01_task-Rest_acq-StimOff_run-01_channels.tsv.
Reading in coordinate system frame Other: None.
Reading electrode coords from /Users/alidzaye/rawdata/sub-003/ses-EphysMedOff01/ieeg/sub-003_ses-EphysMedOff01_space-MNI152NLin2009bAsym_electrodes.tsv.
The read in electrodes file is: 
 [('name', ['ECOG_L_1_SMC_AT', 'ECOG_L_2_SMC_AT', 'ECOG_L_3_SMC_AT', 'ECOG_L_4_SMC_AT', 'ECOG_L_5_SMC_AT', 'ECOG_L_6_SMC_AT', 'LFP_R_1_STN_MT', 'LFP_R_2_STN_MT', 'LFP_R_3_STN_MT', 'LFP_R_4_STN_MT', 'LFP_L_1_STN_MT', 'LFP_L_2_STN_MT', 'LFP_L_3_STN_MT', 'LFP_L_4_STN_MT']), ('x', ['-23.5', '-25.5', '-27.5', '-28.0', '-28.0', '-28.0', '12.3329', '13.2073', '14.0552', '14.8684', '-10.4793', '-11.157', '-11.798', '-12.4066']), ('y', [

<ipython-input-27-cf59c558113f>:27: RuntimeWarning: Did not find any events.tsv associated with sub-003_ses-EphysMedOff01_task-Rest_acq-StimOff_run-01.

The search_str was "/Users/alidzaye/rawdata/sub-003/**/sub-003_ses-EphysMedOff01*events.tsv"
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-27-cf59c558113f>:27: RuntimeWarning: Defaulting coordinate frame to unknown from coordinate system input Other
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-27-cf59c558113f>:27: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_arr = mne_bids.read_raw_bids(bids_path)
<ipython-input-27-cf59c558113f>:27: RuntimeWarning: DigMontage is only a subset of info. There are 2 channel positions not present in the DigMontage. The required channels are:

['EEG_Cz_TM', 'EEG_Fz_TM'].

Consider using inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unkno

ValueError: not enough values to unpack (expected 4, got 1)

In [128]:
read_Bids_files.

[]

In [103]:
raw.get_channel_types()

['eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'eeg',
 'ecog',
 'ecog',
 'ecog',
 'ecog',
 'ecog',
 'ecog',
 'eeg',
 'eeg',
 'emg',
 'emg',
 'ecg',
 'misc',
 'misc',
 'misc',
 'misc',
 'misc',
 'misc']

In [7]:
raw.pick_types(ecog=True).get_channel_types()

['ecog', 'ecog', 'ecog', 'ecog', 'ecog', 'ecog']

In [8]:
raw.pick_types(ecog=True).ch_names

['ECOG_L_1_SMC_AT',
 'ECOG_L_2_SMC_AT',
 'ECOG_L_3_SMC_AT',
 'ECOG_L_4_SMC_AT',
 'ECOG_L_5_SMC_AT',
 'ECOG_L_6_SMC_AT']